In [1]:
import os
import gymnasium as gym
import numpy as np

import ray
from ray.tune.registry import register_env
from ray.rllib.algorithms import ppo, dqn, appo, impala
from ray.rllib.policy.policy import PolicySpec

2023-08-16 11:10:18,728	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
2023-08-16 11:10:18,866	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
2023-08-16 11:10:18,871	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!


In [2]:
from rl.envs.othello import OthelloEnv
from rl.agents.random_policy import OthelloRandomPolicy

In [3]:
ray.init(include_dashboard=False)

2023-08-16 11:10:21,297	INFO worker.py:1621 -- Started a local Ray instance.


Python version:,3.8.17
Ray version:,2.6.2


In [4]:
register_env("othello", lambda _: OthelloEnv({}))

In [5]:
model_config = {
    "fcnet_hiddens": [512, 512],
}

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    agent_id = int(agent_id[-1]) - 1
    return "agent_1" if episode.episode_id % 2 == agent_id else "agent_2"

config = ppo.PPOConfig().environment("othello").framework("torch").rollouts(num_rollout_workers=os.cpu_count() // 2).resources(num_gpus=0)
config = config.multi_agent(policies={"agent_1": PolicySpec(), "agent_2": PolicySpec(policy_class=OthelloRandomPolicy)}, policy_mapping_fn=policy_mapping_fn, policies_to_train=["agent_1"])
config = config.training(model=model_config, _enable_learner_api=False)
config = config.rl_module(_enable_rl_module_api=False)
# config = config.training(num_sgd_iter=10, model={"conv_filters": [[32, [3, 3], 1], [64, [3, 3], 1]]})
# config = config.training(model={"conv_filters": [[32, [3, 3], 2], [64, [3, 3], 2]]}, _enable_learner_api=False)

2023-08-16 11:10:23,120	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-08-16 11:10:23,121	WARNING algorithm_config.py:2572 -- Setting `exploration_config={'type': 'StochasticSampling'}` because you set `_enable_rl_module_api=False`. This exploration config was restored from a prior exploration config that was overriden when setting `_enable_rl_module_api=True`. This occurs because when RLModule API are enabled, exploration_config can not be set.


In [6]:
# import random
# # multiagent environment for othello
# env = OthelloEnv({})
# obs, _ = env.reset()
# terminated = False
# current_player = "agent_1"
# while not terminated:
#     env.render()
#     # action = env.action_space.sample()
#     valid_actions = env.get_valid_moves(current_player)
#     print(current_player, obs.keys())
#     print(obs[current_player].shape)
#     action = algo.compute_single_action(obs[current_player], policy_id=current_player)
#     print(action)
#     if len(valid_actions) > 0:
#         action = random.choice(valid_actions)
#     else:
#         action = 64
#     action = {current_player: action}
#     obs, reward, terminated, truncated, _= env.step(action)
#     reward = reward[current_player]
#     terminated = terminated[current_player]
#     truncated = truncated[current_player]
#     current_player = env.current_player
#     print(terminated, truncated, reward)

In [7]:
from ray import air
from ray import tune

results = tune.Tuner(
    "PPO",
    param_space=config,
    run_config=air.RunConfig(
        checkpoint_config=air.CheckpointConfig(
            checkpoint_at_end=True,
            checkpoint_frequency=50,
        )
    ),
).fit()

2023-08-16 11:10:23,294	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-08-16 11:10:23,295	WARNING algorithm_config.py:2572 -- Setting `exploration_config={'type': 'StochasticSampling'}` because you set `_enable_rl_module_api=False`. This exploration config was restored from a prior exploration config that was overriden when setting `_enable_rl_module_api=True`. This occurs because when RLModule API are enabled, exploration_config can not be set.
(pid=24968) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=24968) 2023-08-16 11:10:27,327	WARNING algorithm_config

In [ ]:
results

(PPO pid=24764) 2023-08-16 10:33:36,334	ERROR actor_manager.py:500 -- Ray error, taking actor 1 out of service. The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=26568, ip=127.0.0.1, actor_id=5c67fffa69c39dc7209aaf9b01000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x0000027092C78B50>)
(PPO pid=24764)   File "c:\Users\nYoshiaki\Documents\rl-mini-app\rl\envs\othello.py", line 64, in step
(PPO pid=24764)     action_dict[self.current_player], terminated, truncated
(PPO pid=24764) KeyError: 'agent_1'
(PPO pid=24764) 
(PPO pid=24764) The above exception was the direct cause of the following exception:
(PPO pid=24764) 
(PPO pid=24764) ray::RolloutWorker.__init__() (pid=26568, ip=127.0.0.1, actor_id=5c67fffa69c39dc7209aaf9b01000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x0000027092C78B50>)
(PPO pid=24764)   File "c:\Users\nYoshiaki\AppData\Local\miniconda3\envs\rl\lib\site-packages\ray\rlli

ResultGrid<[
  Result(
    error='TuneError',
    metrics={'trial_id': 'e596e_00000'},
    path='c://\\Users\\nYoshiaki\\ray_results\\PPO\\PPO_othello_e596e_00000_0_2023-08-16_10-33-26',
    checkpoint=None
  )
]>

(RolloutWorker pid=19464) 2023-08-16 10:33:36,300	WARNING multi_agent_env.py:169 -- observation_space_contains() of <OthelloEnv instance> has not been implemented. You can either implement it yourself or bring the observation space into the preferred format of a mapping from agent ids to their individual observation spaces. 
(RolloutWorker pid=19464) 2023-08-16 10:33:36,300	WARNING multi_agent_env.py:169 -- observation_space_contains() of <OthelloEnv instance> has not been implemented. You can either implement it yourself or bring the observation space into the preferred format of a mapping from agent ids to their individual observation spaces. 
(RolloutWorker pid=19464) 2023-08-16 10:33:36,301	WARNING multi_agent_env.py:169 -- observation_space_contains() of <OthelloEnv instance> has not been implemented. You can either implement it yourself or bring the observation space into the preferred format of a mapping from agent ids to their individual observation spaces. 
(RolloutWorker pid=

In [ ]:
checkpoint = results.get_best_result().checkpoint
print(checkpoint)

None


In [ ]:
import ray
ray.shutdown()
register_env("othello", lambda _: OthelloEnv({}))

In [ ]:
config.expolore = False
algo = config.build()
algo.restore(checkpoint)

2023-08-16 10:33:39,508	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
c:\Users\nYoshiaki\AppData\Local\miniconda3\envs\rl\lib\site-packages\ray\rllib\algorithms\algorithm.py:484: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
c:\Users\nYoshiaki\AppData\Local\miniconda3\envs\rl\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in

ValueError: Traceback (most recent call last):
  File "c:\Users\nYoshiaki\AppData\Local\miniconda3\envs\rl\lib\site-packages\ray\rllib\utils\pre_checks\env.py", line 363, in check_multiagent_environments
    results = env.step(sampled_action)
  File "c:\Users\nYoshiaki\Documents\rl-mini-app\rl\envs\othello.py", line 64, in step
    action_dict[self.current_player], terminated, truncated
KeyError: 'agent_1'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\nYoshiaki\AppData\Local\miniconda3\envs\rl\lib\site-packages\ray\rllib\utils\pre_checks\env.py", line 81, in check_env
    check_multiagent_environments(env)
  File "c:\Users\nYoshiaki\AppData\Local\miniconda3\envs\rl\lib\site-packages\ray\rllib\utils\pre_checks\env.py", line 368, in check_multiagent_environments
    raise ValueError(
ValueError: Your environment (<OthelloEnv instance>) does not abide to the new gymnasium-style API!
From Ray 2.3 on, RLlib only su

In [ ]:
algo.get_policy("agent_1").export_model("othello_policy", onnx=18)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
import random
# multiagent environment for othello
env = OthelloEnv({})
obs, _ = env.reset()
terminated = False
current_player = "agent_1"
while not terminated:
    env.render()
    # action = env.action_space.sample()
    valid_actions = env.get_valid_moves(current_player)
    action = algo.compute_single_action(obs[current_player], policy_id=current_player)
    # if len(valid_actions) > 0:
    #     action = random.choice(valid_actions)
    # else:
    #     action = 64
    action = {current_player: action}
    print(action)
    obs, reward, terminated, truncated, _= env.step(action)
    reward = reward[current_player]
    terminated = terminated[current_player]
    truncated = truncated[current_player]
    current_player = env.current_player

.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|O|X|.|.|.
.|.|.|X|O|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.

{'agent_1': 29}
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|O|O|O|.|.
.|.|.|X|O|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.



TypeError: object of type 'int' has no len()

In [ ]:
env.render()

.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|O|O|O|.|.
.|.|.|X|X|X|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.

